In [ ]:
# 2019, 2020, 2021 모두 한꺼번에 돌리는 것 

전체 한번에 돌리는 코드 

In [ ]:
# 패키지 정리 

import pandas as pd
import numpy as np
from tqdm import tqdm

# okt 
! pip install konlpy
import konlpy
from konlpy.tag import Okt

# 동시등장행렬 라이브러리

import numpy as np
import nltk
from nltk import bigrams
import itertools
import pandas as pd


# 코랩기준 폰트 안깨지게 
# https://teddylee777.github.io/colab/colab-korean

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='NanumBarunGothic') 

# 드라이브 마운트
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
# Google colab: 데이터 경로 설정
path = '/content/gdrive/My Drive/endustrial_engineering_conference_project'

# 파일 읽기
data = pd.read_csv(path + '/pre_finished_data/final_data.csv', index_col = 0)
data.tail()

# 날짜 분류 
# 특정 기간의 데이터 추출하기 -> object type 일때 사용
data_2019 = data[(data["date"] >= '2019-01-01') & (data["date"] <= '2019-12-31')].reset_index(drop = True)
data_2019

data_2020 = data[(data["date"] >= '2020-01-01') & (data["date"] <= '2020-12-31')].reset_index(drop = True)
data_2020

data_2021 = data[(data["date"] >= '2021-01-01') & (data["date"] <= '2021-12-31')].reset_index(drop = True)
data_2021

for index,year_data in enumerate([data_2019,data_2020,data_2021]):
    data = year_data.copy()

    # 한글만 추출 -> 특수문자 제거
    data['content'] = data['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    # print(data['content'].iloc[:2])

    # 한글과 공백 제외하고 모두 제거하면서 생겼을 수도 있는 null 값 확인
    data['content'].replace('', np.nan, inplace=True)
    data = data.dropna()
    # print(data.isnull().sum())

    
    # 형태소 분석, 명사 추출 

    okt = Okt()

    def func(data):

      # 형태소 분석
      clean = []
      for word in okt.pos(data, stem=True): #어간 추출
          if word[1] in ['Noun', 'Adjective']: # 명사, 형용사 추출
              clean.append(word[0])

      return " ".join(clean) 

    data['content'] = data['content'].apply(lambda x : func(x))
    print(data['content'].head())   


    def preprocessing(content):

        stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','요','대한','대해','안','회원','쪽','있습니다','글','생각','준비','업','독','취사','있을까요','조금','눈','네이버','공부','과정',
    '자','에','와','한','하다','그리고','넘','네','을','랑','예요','오','여서','사람','여러분','진짜','학년','관련','및','그냥','같습니다','리스트','부분','없습니다','저작권','등','정도','가야','부탁드립니다'
    ,'이에요','데','것','그','저','더','같아요','근데','듯','입니다','명','댓글','있는','이상','없는','다시','건','관리자','이야기','전','가요','이런','식','이후','산업','학과','공학','영상','진행','학교','어디','문제',
    '에게','에서','라서','이라서','에요','와','만','나','로','이랑','내','엔','아','부터','수','안녕하세요','도움','현재','있어요','어떻게','추천','좋은','다른','학생','무료','필요','대학교','대학','전공','클릭',
    '때','거','다','이다','이나','에도','것','고','게','인데','제','까지','에는','엔','이라','작성자','게시','고민','조언','비슷한','라면','어떤','본인','먼저','지금','독취','저작','카페','이유','콘텐츠','감사',
    '약간','오늘','점','없이','자꾸','알','있','다니','또','어','딱','걍','더','중','니','저','면','듯','분야','같은','시스템','경우','시간','소개','많은','내용','있고','있으며','때문','통해','정말']

        # 문장 리스트 
        sentence_data = []
        for sentence in tqdm(content):
            temp_X = []
            temp_X = okt.morphs(sentence, stem=True)  # 토큰화
            temp_X = [word for word in temp_X if not word in stopwords]  # 불용어 제거
            sentence_data.append(temp_X)

        # 모든 어휘 리스트 
        word_data = list(itertools.chain.from_iterable(sentence_data))
        

        return sentence_data,word_data

    sentence_data,word_data = preprocessing(data['content'])
    print(sentence_data[0])
    print(word_data[0])

    # 동시등장행렬 생성 함수
    def generate_co_occurrence_matrix(corpus):
        vocab = set(corpus)
        vocab = list(vocab)
        vocab_index = {word: i for i, word in enumerate(vocab)}
    
        # Create bigrams from all words in corpus
        bi_grams = list(bigrams(corpus))
    
        # Frequency distribution of bigrams ((word1, word2), num_occurrences)
        bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))
    
        # Initialise co-occurrence matrix
        # co_occurrence_matrix[current][previous]
        co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))
    
        # Loop through the bigrams taking the current and previous word,
        # and the number of occurrences of the bigram.
        for bigram in bigram_freq:
            current = bigram[0][1]
            previous = bigram[0][0]
            count = bigram[1]
            pos_current = vocab_index[current]
            pos_previous = vocab_index[previous]
            co_occurrence_matrix[pos_current][pos_previous] = count
        co_occurrence_matrix = np.matrix(co_occurrence_matrix)
    
        # return the matrix and the index
        return co_occurrence_matrix, vocab_index


    # 문장 별로 저장
    # 예시, sentence_data = [['Where', 'Python', 'is', 'used'],['What', 'is', 'Python' 'used', 'in'],['Why', 'Python', 'is', 'best'],['What', 'companies', 'use', 'Python']] # fl

    # 여러 리스트 하나도 합치기 
    # 예시, word_data = ['Where', 'Python', 'is', 'used','What', 'is', 'Python' 'used', 'in','Why', 'Python', 'is', 'best','What', 'companies', 'use', 'Python']

    matrix, vocab_index = generate_co_occurrence_matrix(word_data)
    # matrix, vocab_index


    data_matrix = pd.DataFrame(matrix, index=vocab_index,
                                columns=vocab_index)
    data_matrix

    filename = f'co_occurrence_total_{index+2019}_noun_adjacent'
    data_matrix.to_csv(path + '/network_anaysis/co_occurrence_matrix/' + filename + '.csv', sep = ',',encoding='utf-8-sig')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: FutureWarning: The default value of regex will change from True to False in a future version.


1                                    여기 산 경이 산업 학과 아니다
2    산 공이 애매하다 기사 대리 과장 건너 띠 부장 이상 필요하다 전공 라면 과연 기사...
3    안녕하십니까 아태 물류 학부 학번 재학 물류 학도 이다 최근 과 최적화 수업 그 분...
4    급격하다 기술 혁신 차 산업혁명 기존 산업 구조 변화 인공 지능 사물인터넷 로봇공학...
5                              필 이모티콘 자기 합리화 기네 이모티콘 넌
Name: content, dtype: object


100%|██████████| 2045/2045 [00:05<00:00, 350.03it/s]


['여기', '산', '경이', '아니다']
여기
0                                        품질 관리 연구 주제
1                           관련 고민 대상 이야기 방 이다 편하다 정보
2                           인 서울 중 하위 산업 학과 진로 고민 이다
3                                    산 공이 학부 수준 유리하다
4    산업 학과 전공 이산 수산 없다 제 경험 생산 관리 프로그래밍 때 조금 도움 것 같다
Name: content, dtype: object


100%|██████████| 2400/2400 [00:10<00:00, 224.67it/s]


['품질', '관리', '연구', '주제']
품질
0                                                약대 수의
1                         학점은행제 산업 학과 전문대 산업 경영 졸업 후 법
2                                                     
3                                                    네
4    일단 급여 차이 심하다 네트워크 해도 비젼 없다 갈곳도 별로 없다 개발 쪽 회사 소...
Name: content, dtype: object


100%|██████████| 3156/3156 [00:18<00:00, 172.38it/s]


['약대', '수의']
약대
